# HOMEWORK 11

In this homework you are going to improve the DNN for house price estimation we built during the lesson. The objective is to beat the performance (i.e. to get a closer estimation) of the last network we built in the lecture.

The structure of this homework is as follows:
 * First, we are going to build the reference network (exactly the same network we implemented in the lecture).
 * You are going to run it a write down the price estimation error (Validation MAE).
 * Then you are going to make some modifications in order to make the network perform better.
 
In the section **Tasks** you will find what modification you'll need to implement as well as some hints.
 
At the end, as usual, there are some question waiting for you to answer :-)

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

from tensorflow.keras import Model, metrics
from tensorflow.keras.layers import Dense, Dropout, Input

# Set the seeds
from numpy.random import seed
from tensorflow.random import set_seed
seed_value = 1234578790
seed(seed_value)
set_seed(seed_value)

### Data Preparation

Let's load the data and prepare the training sets.

In [15]:
dataset = pd.read_csv('./data/train.csv')

features = ['SalePrice','OverallQual', 'GrLivArea', 'GarageCars', 'FullBath', 'YearBuilt', 'MSSubClass', 'LotFrontage', 'LotArea', 'MoSold', 'YrSold']
data = dataset[features]

# Filling nan with the mean of the column:
data = data.fillna(data.mean())

# Extract input values and normalize
x = data[features[1:]]
scale = StandardScaler()
x = scale.fit_transform(x)

# Extract output values (prices) and normalize
y = data[features[0]].values
y = y/100000

# Split into 75% for train and 25% for test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=seed_value)

# dataset
data

,SalePrice,OverallQual,GrLivArea,GarageCars,FullBath,YearBuilt,MSSubClass,LotFrontage,LotArea,MoSold,YrSold
0,208500,7,1710,2,2,2003,60,65.0,8450,2,2008
1,181500,6,1262,2,2,1976,20,80.0,9600,5,2007
2,223500,7,1786,2,2,2001,60,68.0,11250,9,2008
3,140000,7,1717,3,1,1915,70,60.0,9550,2,2006
4,250000,8,2198,3,2,2000,60,84.0,14260,12,2008
...,...,...,...,...,...,...,...,...,...,...,...
1455,175000,6,1647,2,2,1999,60,62.0,7917,8,2007
1456,210000,6,2073,2,2,1978,20,85.0,13175,2,2010
1457,266500,7,2340,1,2,1941,70,66.0,9042,5,2010
1458,142125,5,1078,1,1,1950,20,68.0,9717,4,2010


Let's use the same plotting helper we used in the lecture.

In [16]:
def plot_history(history):
    h = history.history
    epochs = range(len(h['loss']))

    plt.subplot(121), plt.plot(epochs, h['loss'], '.-', epochs, h['val_loss'], '.-')
    plt.grid(True), plt.xlabel('epochs'), plt.ylabel('loss')
    plt.legend(['Train', 'Validation'])
    plt.subplot(122), plt.plot(epochs, np.array(h['mean_absolute_error'])*1e5, '.-',
                               epochs, np.array(h['val_mean_absolute_error'])*1e5, '.-')
    plt.grid(True), plt.xlabel('epochs'), plt.ylabel('MAE')
    plt.legend(['Train', 'Validation'])
        
    print('Train MAE     ', h['mean_absolute_error'][-1]*1e5)
    print('Validation MAE', h['val_mean_absolute_error'][-1]*1e5)

### Building the Network

In [45]:
inputs = Input(shape=x.shape[1])
outputs = Dense(32, activation='relu')(inputs)
outputs = Dense(32, activation='relu')(outputs)
outputs = Dense(16, activation='relu')(outputs)
outputs = Dense(8, activation='relu')(outputs)
outputs = Dense(1, activation='linear')(outputs)
model = Model(inputs, outputs)

model.compile(optimizer ='adam', loss ='mean_squared_error', metrics =[metrics.mae])
model.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 10)]              0         
                                                                 
 dense_42 (Dense)            (None, 32)                352       
                                                                 
 dense_43 (Dense)            (None, 32)                1056      
                                                                 
 dense_44 (Dense)            (None, 16)                528       
                                                                 
 dense_45 (Dense)            (None, 8)                 136       
                                                                 
 dense_46 (Dense)            (None, 1)                 9         
                                                                 
Total params: 2,081
Trainable params: 2,081
Non-trainable 

### Training the Network

In [46]:
history = model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=300, batch_size=32, verbose=1)
plot_history(history)

Epoch 1/300
35/35 [==============================] - 1s 8ms/step - loss: 1.2921 - mean_absolute_error: 0.8899 - val_loss: 0.8265 - val_mean_absolute_error: 0.5947
Epoch 2/300
35/35 [==============================] - 0s 6ms/step - loss: 0.4455 - mean_absolute_error: 0.4782 - val_loss: 0.4413 - val_mean_absolute_error: 0.4285
Epoch 3/300
35/35 [==============================] - 0s 6ms/step - loss: 0.2963 - mean_absolute_error: 0.3877 - val_loss: 0.3094 - val_mean_absolute_error: 0.3667
Epoch 4/300
35/35 [==============================] - 0s 6ms/step - loss: 0.2324 - mean_absolute_error: 0.3429 - val_loss: 0.2395 - val_mean_absolute_error: 0.3285
Epoch 5/300
35/35 [==============================] - 0s 7ms/step - loss: 0.1924 - mean_absolute_error: 0.3071 - val_loss: 0.2010 - val_mean_absolute_error: 0.3042
Epoch 6/300
35/35 [==============================] - 0s 7ms/step - loss: 0.1738 - mean_absolute_error: 0.2907 - val_loss: 0.1884 - val_mean_absolute_error: 0.3005
Epoch 7/300
35/35 [===

35/35 [==============================] - 0s 5ms/step - loss: 0.0539 - mean_absolute_error: 0.1695 - val_loss: 0.1362 - val_mean_absolute_error: 0.2455
Epoch 52/300
35/35 [==============================] - 0s 6ms/step - loss: 0.0511 - mean_absolute_error: 0.1613 - val_loss: 0.1253 - val_mean_absolute_error: 0.2446
Epoch 53/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0515 - mean_absolute_error: 0.1657 - val_loss: 0.1373 - val_mean_absolute_error: 0.2450
Epoch 54/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0510 - mean_absolute_error: 0.1620 - val_loss: 0.1287 - val_mean_absolute_error: 0.2494
Epoch 55/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0472 - mean_absolute_error: 0.1544 - val_loss: 0.1333 - val_mean_absolute_error: 0.2416
Epoch 56/300
35/35 [==============================] - 0s 6ms/step - loss: 0.0470 - mean_absolute_error: 0.1529 - val_loss: 0.1306 - val_mean_absolute_error: 0.2504
Epoch 57/300
35/35 [=========

35/35 [==============================] - 0s 6ms/step - loss: 0.0322 - mean_absolute_error: 0.1295 - val_loss: 0.1466 - val_mean_absolute_error: 0.2649
Epoch 102/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0304 - mean_absolute_error: 0.1264 - val_loss: 0.1329 - val_mean_absolute_error: 0.2466
Epoch 103/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0279 - mean_absolute_error: 0.1189 - val_loss: 0.1375 - val_mean_absolute_error: 0.2482
Epoch 104/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0281 - mean_absolute_error: 0.1185 - val_loss: 0.1426 - val_mean_absolute_error: 0.2521
Epoch 105/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0276 - mean_absolute_error: 0.1201 - val_loss: 0.1391 - val_mean_absolute_error: 0.2501
Epoch 106/300
35/35 [==============================] - 0s 4ms/step - loss: 0.0282 - mean_absolute_error: 0.1232 - val_loss: 0.1632 - val_mean_absolute_error: 0.2595
Epoch 107/300
35/35 [===

Epoch 151/300
35/35 [==============================] - 0s 6ms/step - loss: 0.0200 - mean_absolute_error: 0.1047 - val_loss: 0.1448 - val_mean_absolute_error: 0.2552
Epoch 152/300
35/35 [==============================] - 0s 6ms/step - loss: 0.0204 - mean_absolute_error: 0.1045 - val_loss: 0.1476 - val_mean_absolute_error: 0.2578
Epoch 153/300
35/35 [==============================] - 0s 6ms/step - loss: 0.0194 - mean_absolute_error: 0.1025 - val_loss: 0.1505 - val_mean_absolute_error: 0.2634
Epoch 154/300
35/35 [==============================] - 0s 6ms/step - loss: 0.0206 - mean_absolute_error: 0.1057 - val_loss: 0.1609 - val_mean_absolute_error: 0.2631
Epoch 155/300
35/35 [==============================] - 0s 6ms/step - loss: 0.0215 - mean_absolute_error: 0.1078 - val_loss: 0.1476 - val_mean_absolute_error: 0.2636
Epoch 156/300
35/35 [==============================] - 0s 6ms/step - loss: 0.0201 - mean_absolute_error: 0.1052 - val_loss: 0.1538 - val_mean_absolute_error: 0.2695
Epoch 157/

Epoch 201/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0156 - mean_absolute_error: 0.0920 - val_loss: 0.1419 - val_mean_absolute_error: 0.2650
Epoch 202/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0163 - mean_absolute_error: 0.0951 - val_loss: 0.1496 - val_mean_absolute_error: 0.2658
Epoch 203/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0165 - mean_absolute_error: 0.0947 - val_loss: 0.1478 - val_mean_absolute_error: 0.2672
Epoch 204/300
35/35 [==============================] - 0s 6ms/step - loss: 0.0154 - mean_absolute_error: 0.0913 - val_loss: 0.1453 - val_mean_absolute_error: 0.2612
Epoch 205/300
35/35 [==============================] - 0s 6ms/step - loss: 0.0156 - mean_absolute_error: 0.0930 - val_loss: 0.1441 - val_mean_absolute_error: 0.2639
Epoch 206/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0153 - mean_absolute_error: 0.0909 - val_loss: 0.1520 - val_mean_absolute_error: 0.2738
Epoch 207/

35/35 [==============================] - 0s 6ms/step - loss: 0.0111 - mean_absolute_error: 0.0769 - val_loss: 0.1440 - val_mean_absolute_error: 0.2641
Epoch 251/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0119 - mean_absolute_error: 0.0803 - val_loss: 0.1460 - val_mean_absolute_error: 0.2681
Epoch 252/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0141 - mean_absolute_error: 0.0882 - val_loss: 0.1484 - val_mean_absolute_error: 0.2667
Epoch 253/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0133 - mean_absolute_error: 0.0856 - val_loss: 0.1468 - val_mean_absolute_error: 0.2655
Epoch 254/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0128 - mean_absolute_error: 0.0845 - val_loss: 0.1488 - val_mean_absolute_error: 0.2643
Epoch 255/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0123 - mean_absolute_error: 0.0810 - val_loss: 0.1475 - val_mean_absolute_error: 0.2681
Epoch 256/300
35/35 [===

Epoch 300/300
35/35 [==============================] - 0s 5ms/step - loss: 0.0095 - mean_absolute_error: 0.0718 - val_loss: 0.1602 - val_mean_absolute_error: 0.2746
Train MAE      7180.623710155487
Validation MAE 27464.80405330658


### Tasks

Improve the network so you obtain a better estimation error than the reference. You can try the following:
* Add a new feature to your inputs. For instance, `LotArea` (lot size in square feet) seems like a good candidate :-)
* Try increasing the number of neurons in the first layer. Currently, we have 5 neurons there, what happens if we increase it to 10?

### Questions
* What is the new price estimation error after your modifrications? Is it better than before?

**Best train results with 4 layers (2, 32, 16, 8 neurons) and 300 epochs**<br>
**Train MAE      7180.623710155487**<br>
**Validation MAE 27464.80405330658**

**Best validation results with 3 layers (16, 8, 4 neurons) and 150 epochs**<br>
**Train MAE      18225.429952144623**<br>
**Validation MAE 21605.081856250763**

* Does adding more features help?

**Yes, it help**

* Does adding more neurons help?

**It's also help**

* What would you do to improv the network even further?

**Proceed with experementing numbers of layers and neurons. I'm not sure that other optimizator and loss function can help to improve results.**

In [47]:
'''
ALL EXPIREMENRS

Initial results
Train MAE      22759.780287742615
Validation MAE 25656.235218048096

Add 'MSSubClass', 'LotFrontage', 'LotArea', 'MoSold', 'YrSold' - 150 epochs
Train MAE      22799.111902713776
Validation MAE 24299.943447113037

One layer with 11 neurons - 150 epochs
Train MAE      20802.69604921341
Validation MAE 23955.920338630676

Two layers with 5 neurons - 150 epochs
Train MAE      19456.45809173584
Validation MAE 23226.720094680786

Two layers with 10 and 5 neurons - 150 epochs
Train MAE      19268.60362291336
Validation MAE 23579.780757427216

Two layers with 10 neurons - 150 epochs
Train MAE      19547.033309936523
Validation MAE 23579.45740222931

Two layers with 5 and 10 neurons - 150 epochs
Train MAE      21481.90289735794
Validation MAE 22874.57436323166

Three layers with 10, 5 and 5 neurons - 150 epochs
Train MAE      20146.171748638153
Validation MAE 24146.375060081482

Three layers with 10, 5 and 3 neurons - 150 epochs
Train MAE      19770.529866218567
Validation MAE 22671.140730381012

Three layers with 16, 8 and 4 neurons - 150 epochs
Train MAE      18225.429952144623
Validation MAE 21605.081856250763

Three layers with 16, 8 and 4 neurons - 200 epochs
Train MAE      18203.885853290558
Validation MAE 22975.045442581177

Three layers with 32, 16, 8 neurons - 200 epochs
Train MAE      13096.493482589722
Validation MAE 24769.487977027893

Three layers with 32, 16, 8 neurons - 300 epochs
Train MAE      12026.68771147728
Validation MAE 26279.088854789734

Three layers with 32, 16, 8 neurons - 300 epochs
Train MAE      12026.68771147728
Validation MAE 26279.088854789734

Four layers with 32, 32, 16, 8 neurons - 300 epochs
Train MAE      7180.623710155487
Validation MAE 27464.80405330658
'''

"\nALL EXPIREMENRS\n\nInitial results\nTrain MAE      22759.780287742615\nValidation MAE 25656.235218048096\n\nAdd 'MSSubClass', 'LotFrontage', 'LotArea', 'MoSold', 'YrSold' - 150 epochs\nTrain MAE      22799.111902713776\nValidation MAE 24299.943447113037\n\nOne layer with 11 neurons - 150 epochs\nTrain MAE      20802.69604921341\nValidation MAE 23955.920338630676\n\nTwo layers with 5 neurons - 150 epochs\nTrain MAE      19456.45809173584\nValidation MAE 23226.720094680786\n\nTwo layers with 10 and 5 neurons - 150 epochs\nTrain MAE      19268.60362291336\nValidation MAE 23579.780757427216\n\nTwo layers with 10 neurons - 150 epochs\nTrain MAE      19547.033309936523\nValidation MAE 23579.45740222931\n\nTwo layers with 5 and 10 neurons - 150 epochs\nTrain MAE      21481.90289735794\nValidation MAE 22874.57436323166\n\nThree layers with 10, 5 and 5 neurons - 150 epochs\nTrain MAE      20146.171748638153\nValidation MAE 24146.375060081482\n\nThree layers with 10, 5 and 3 neurons - 150 epo